In [ ]:

import pandas as pd
import os
import shutil
from os import path as osp
from src.intensity import add_features_and_create_new_dicts


BASEDIR = '/home/amrit/data/proj_data/MLG_project/DLBCL-Morph'


STAIN = 'MYC'
FIDIR = f'{BASEDIR}/outputs'
CLINPATH = f'{BASEDIR}/clinical_data_cleaned.csv'
ANNPATH = f'{BASEDIR}/annotations_clean.csv'
FEATSDIR = f'{BASEDIR}/outputs/files/{STAIN}'
FEATSCALERPATH = f"{FEATSDIR}/0_feat_scaler.npz"
PATCH_SIZE = 224
OUTPUT_SIZE = PATCH_SIZE*8

##########
# read annotation csv, filter, and process intensity features
df = pd.read_csv(ANNPATH)

df = df[df['stain'] == STAIN]
df['area'] = (df['xe'] - df['xs']) *  (df['ye'] - df['ys'])/10000
df = df[df['area'] >= 150]  
df = df[df['xs']  >=0 ]
df = df[df['ys']  >=0 ]
df = df[df['xe']  >=0 ]
df = df[df['ye']  >=0 ]

df = df.reset_index()
##########

###############
# add intensity features
start_index = 0
end_index = len(df.index)

datpaths = []
imgpaths = []
updatpaths = []


In [ ]:
# for index in range(start_index, 1):
for index in range(start_index, end_index):
    df_index = df['index'][index]
    patient_id = df['patient_id'][index]
    stain = df['stain'][index]
    tma_id = df['tma_id'][index]
    unique_id = str(patient_id) + '_' + stain + '_' + str(df_index)

    img_file_name = f"{FIDIR}/images/{patient_id}/{patient_id}_{stain}_{tma_id}_{OUTPUT_SIZE}_{df_index}.png"
    dat_file_name = f"{FIDIR}/files/{stain}/{patient_id}/{df_index}/0.dat"
    updat_file_name = f"{FIDIR}/files/{stain}/{patient_id}/{df_index}/{unique_id}.dat"

    datpaths.append(dat_file_name)
    imgpaths.append(img_file_name)
    updatpaths.append(updat_file_name)
    

In [ ]:

import numpy as np
import joblib
import cv2
from tqdm import tqdm


def get_cell_features(celldata, image_global, saliencymap_global, original=False):
    contour_global = np.array(celldata['contour'])
    if original:
        contour = contour_global
        image = image_global
        saliencymap = saliencymap_global
    else:
        img_max_coord = image_global.shape[:2][::-1]
        # recalc from contour
        cont_min, cont_max = np.min(contour_global, axis=0), np.max(contour_global, axis=0)
        # add padding to allow proper saliency calc because boundary is 3 pixels
        padding = 5
        cont_min = np.clip(cont_min-padding, 0, img_max_coord)
        cont_max = np.clip(cont_max+padding, 0, img_max_coord)

        # inverted because array dims transpose of coordinates
        box_min, box_max = cont_min[::-1], cont_max[::-1]
        # filter cell if bbox is outside image dimensions
        if contour_global.shape[0] < 5:
            return None
        contour = contour_global - np.expand_dims(cont_min, axis=0)
        image = image_global[box_min[0]:box_max[0], box_min[1]:box_max[1], :]
        saliencymap = saliencymap_global[box_min[0]:box_max[0], box_min[1]:box_max[1]]

    # area
    area = cv2.contourArea(contour)
    # solidity
    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    solidity = float(area)/hull_area
    # major/minor axis len
    (x,y),(minAxisLen,majAxisLen),angle = cv2.fitEllipse(contour)
    perimeterLen = cv2.arcLength(contour, True)
    # circularity
    circularity = 4 * np.pi * area / (perimeterLen**2)
    # eccentiricity
    eccentiricity = np.sqrt((majAxisLen)**2 - (minAxisLen)**2) / (majAxisLen)
    # mean intensities
    mask = np.zeros(image[:,:,0].shape, np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, -1)
    maskedimg = image[np.where(mask == 255)]
    intensities = np.mean(maskedimg, axis=0)
        
    # intensity range
    mins = np.min(maskedimg, axis=0)
    maxes = np.max(maskedimg, axis=0)
   
    intensity_range = maxes - mins

    # boundary
    boundary = np.zeros(image[:,:,0].shape, np.uint8)
    # currently the boundary thickness is 3 pixels
    cv2.drawContours(boundary, [contour], -1, 255, 3)

    # boundary intensity
    boundaryimg = image[np.where(boundary == 255)]
    boundary_intensities = np.mean(boundaryimg, axis=0)

    # boundary saliency
    boundary_saliencies = saliencymap[np.where(boundary == 255)]
    boundary_saliency_mean = np.mean(boundary_saliencies)

    # return {
    # 	"area" : area,
    # 	"eccentiricity" : eccentiricity,
    # 	"solidity" : solidity,
    # 	"circularity" : circularity,
    # 	"majorAxisLength" : majAxisLen,
    # 	"minAxisLength" : minAxisLen,
    # 	"blue_mean_intensity" : intensities[0],
    # 	"green_mean_intensity" : intensities[1],
    # 	"red_mean_intensity" : intensities[2],
    # 	"blue_intensity_range" : intensity_range[0],
    # 	"green_intensity_range" : intensity_range[1],
    # 	"red_intensity_range" : intensity_range[2],
    # 	"blue_boundary_intensity" : boundary_intensities[0],
    # 	"green_boundary_intensity" : boundary_intensities[1],
    # 	"red_boundary_intensity" : boundary_intensities[2],
    # 	"boundary_saliency" : boundary_saliency_mean,
    # }

    return np.array([
        area,
        eccentiricity,
        solidity,
        circularity,majAxisLen,
        minAxisLen,
        intensities[0],
        intensities[1],
        intensities[2],
        intensity_range[0],
        intensity_range[1],
        intensity_range[2],
        boundary_intensities[0],
        boundary_intensities[1],
        boundary_intensities[2],
        boundary_saliency_mean,
    ])

saliency = cv2.saliency.StaticSaliencyFineGrained_create()
def get_saliency_map(image):
    _, saliencymap = saliency.computeSaliency(image)
    return saliencymap

def add_features_and_create_new_dicts(datpaths, imgpaths, updatpaths):
    for datpath, imgpath, updatpath in zip(tqdm(datpaths), imgpaths, updatpaths):
        celldata_dict = joblib.load(datpath)
        celldata_list = [(k, v) for k, v in celldata_dict.items()]
        # celldata_list = celldata_list[:100]

        image = cv2.imread(imgpath)
        saliencymap = get_saliency_map(image)

        cellfeat_list = joblib.Parallel(4)(
            joblib.delayed(get_cell_features)(celldata, image, saliencymap)
            for _, celldata in tqdm(celldata_list, disable=True)
        )
        # cellfeat_list = [
        #     get_cell_features(celldata, image, saliencymap)
        #     for _, celldata in tqdm(celldata_list, disable=True)
        # ]

        cellfeat_list_temp = [x for x in cellfeat_list if x is not None]
        valid_cells_n = len(cellfeat_list_temp)
        total_cells_n = len(cellfeat_list)
        valid_cell_frac = (valid_cells_n/total_cells_n)
        
        if valid_cell_frac < 1.0:
            print('updatpath:', updatpath)
            print('datpath:', datpath)
            print('imgpath:', imgpath)
            print(f'valid_cells_frac {valid_cell_frac} = 1-{total_cells_n-valid_cells_n}/{total_cells_n}')

        celldatanew_dict = {
            k: {**v, 'intensity_feats': feats}
            for (k, v), feats in zip(celldata_list, cellfeat_list)
            if feats is not None
        }
        joblib.dump(celldatanew_dict, updatpath)
        

In [8]:
# 20 minutes

# shutil.rmtree(FEATSDIR)
# if not osp.exists(FEATSDIR):
#     os.makedirs(FEATSDIR)
add_features_and_create_new_dicts(datpaths, imgpaths, updatpaths)
###############

# get all cell features in all datapoints and get normalization stats
print('Building global cell feature normalizer')
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import joblib
import numpy as np

 91%|█████████▏| 333/364 [18:26<01:49,  3.52s/it]

updatpath: /home/amrit/data/proj_data/MLG_project/DLBCL-Morph/outputs/files/MYC/13981/2615/13981_MYC_2615.dat
datpath: /home/amrit/data/proj_data/MLG_project/DLBCL-Morph/outputs/files/MYC/13981/2615/0.dat
imgpath: /home/amrit/data/proj_data/MLG_project/DLBCL-Morph/outputs/images/13981/13981_MYC_TA234_1792_2615.png
valid_cells_frac 0.9998890614599512 = 1-1/9014


100%|██████████| 364/364 [20:13<00:00,  3.33s/it]


Building global cell feature normalizer


In [ ]:
# 4 minutes
gns = StandardScaler()
for featpath in tqdm(updatpaths):
    # print(featpath)
    celldatadict = joblib.load(featpath)
    cellsfeats = np.array([v['intensity_feats'] for k, v in celldatadict.items()])
    gns.partial_fit(cellsfeats)

In [10]:
FEATSCALERPATH

'/home/amrit/data/proj_data/MLG_project/DLBCL-Morph/outputs/files/MYC/0_feat_scaler.npz'

In [11]:
np.savez(FEATSCALERPATH, mean=gns.mean_, var=gns.var_)

# dd = np.load(FEATSCALERPATH)
# print(dd['mean'], gns.mean_)
# print(dd['var'], gns.var_)


In [126]:

import pandas as pd
import os
import shutil
from os import path as osp
from src.intensity import add_features_and_create_new_dicts


BASEDIR = '/home/amrit/data/proj_data/MLG_project/DLBCL-Morph'


STAIN = 'MYC'
FIDIR = f'{BASEDIR}/outputs'
CLINPATH = f'{BASEDIR}/clinical_data_cleaned.csv'
ANNPATH = f'{BASEDIR}/annotations_clean.csv'
FEATSDIR = f'{BASEDIR}/outputs/files/{STAIN}'
FEATSCALERPATH = f"{FEATSDIR}/0_feat_scaler.npz"
# PATCH_SIZE = 224
# OUTPUT_SIZE = PATCH_SIZE*8


GRAPHSDIR = f'{BASEDIR}/graphs'
LABELSPATH = f'{BASEDIR}/graphs/0_labels.json'
PATCH_SIZE = 300
SKEW_NOISE = 0.0001
MIN_CELLS_PER_PATCH = 2
CONNECTIVITY_DISTANCE = 500

global_patch_stats = []

# divide into quantiles
df = pd.read_csv(CLINPATH)
df_labels = get_pids_labels_for_key(df, key='OS', nclasses=3)

# # save paths
# featpaths = np.sort(glob.glob(f'{FEATSDIR}/**/*.dat', recursive=True)) #np.sort(glob.glob(f'{FEATSDIR}/*.dat'))
# featpaths = [x if "/0.dat" not in x for x in featpaths]
featpaths = np.sort(glob.glob(f'{FEATSDIR}/**/*.dat', recursive=True)) #np.sort(glob.glob(f'{FEATSDIR}/*.dat'))
featpaths = [x for x in featpaths if ("/0.dat" not in x) and ("/file_map.dat" not in x)]
featpaths
pids = [int(osp.basename(featpath).split('_')[0]) for featpath in featpaths]
df_featpaths = pd.DataFrame(zip(pids, featpaths), columns=['patient_id', 'featpath'])

# merge to find datapoints with graph data and labels
df_data = df_featpaths.merge(df_labels, on='patient_id')
# df_data = df_data[:12]

featpaths_data = df_data['featpath'].to_list()
labels_data = df_data['OS_class'].to_list()
outgraphpaths_data = [f"{GRAPHSDIR}/{osp.basename(featpath).split('.')[0]}.json" for featpath in featpaths_data]

# save labels
labels_dict = {osp.basename(graphpath): label for graphpath, label in zip(outgraphpaths_data, labels_data)}
with open(LABELSPATH, 'w') as f:
    json.dump(labels_dict, f)



In [127]:
LABELSPATH

'/home/amrit/data/proj_data/MLG_project/DLBCL-Morph/graphs/0_labels.json'

In [115]:

# read normalizer stats from file and pass to fn
dd = np.load(FEATSCALERPATH)
cell_feat_norm_stats = (dd['mean'], dd['var'])

# create final graphs data
# shutil.rmtree(GRAPHSDIR)
if not osp.exists(GRAPHSDIR):
    os.makedirs(GRAPHSDIR)
    create_graph_with_pooled_patch_nodes(
        featpaths_data,
        labels_data,
        outgraphpaths_data,
        PATCH_SIZE,
        cell_feat_norm_stats=cell_feat_norm_stats,
    )

    # wont work in parallel mode
    # print(np.mean(global_patch_stats), np.std(global_patch_stats))

  0%|          | 0/319 [00:00<?, ?it/s]

100%|██████████| 319/319 [01:02<00:00,  5.13it/s]


In [152]:

ON_GPU = False

device = select_device(on_gpu=ON_GPU)

SEED = 5
random.seed(SEED)
rng = np.random.default_rng(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)


BASEDIR = '/home/amrit/data/proj_data/MLG_project/DLBCL-Morph'


STAIN = 'MYC'
FIDIR = f'{BASEDIR}/outputs'
CLINPATH = f'{BASEDIR}/clinical_data_cleaned.csv'
ANNPATH = f'{BASEDIR}/annotations_clean.csv'
FEATSDIR = f'{BASEDIR}/outputs/files/{STAIN}'
FEATSCALERPATH = f"{FEATSDIR}/0_feat_scaler.npz"
# PATCH_SIZE = 224
# OUTPUT_SIZE = PATCH_SIZE*8

WORKSPACE_DIR = Path(BASEDIR)
GRAPH_DIR = WORKSPACE_DIR / "graphs"
LABELS_PATH = WORKSPACE_DIR / "graphs/0_labels.txt"

# GRAPHSDIR = f'{BASEDIR}/graphs'
# LABELSPATH = f'{BASEDIR}/graphs/0_labels.json'
PATCH_SIZE = 300
SKEW_NOISE = 0.0001
MIN_CELLS_PER_PATCH = 2
CONNECTIVITY_DISTANCE = 500


# WORKSPACE_DIR = Path("./workspace/ours")
# GRAPH_DIR = WORKSPACE_DIR / "graphs"
# LABELS_PATH = WORKSPACE_DIR / "graphs/0_labels.txt"
NUM_EPOCHS = 100
NUM_NODE_FEATURES = 128
NCLASSES = 3

# TRAIN_DIR = BASEDIR + "/training"
# SPLIT_PATH = TRAIN_DIR + "/splits.dat"
# RUN_OUTPUT_DIR = TRAIN_DIR + f"/session_{datetime.now().strftime('%m_%d_%H_%M_%S')}"
# # os.mkdir(TRAIN_DIR)
# # os.mkdir(RUN_OUTPUT_DIR)
TRAIN_DIR = WORKSPACE_DIR / "training"
SPLIT_PATH = TRAIN_DIR / "splits.dat"
RUN_OUTPUT_DIR = TRAIN_DIR / f"session_{datetime.now().strftime('%m_%d_%H_%M_%S')}"
RUN_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR = RUN_OUTPUT_DIR / "model"

# MODEL_DIR = RUN_OUTPUT_DIR + "/model"

In [153]:
GRAPHSDIR

'/home/amrit/data/proj_data/MLG_project/DLBCL-Morph/graphs'

In [154]:
wsi_paths = recur_find_ext(GRAPHSDIR, [".json"])
wsi_names = [Path(v).stem for v in wsi_paths]
assert len(wsi_paths) > 0, "No files found."  # noqa: S101

len(wsi_paths) , len(wsi_names)

(320, 320)

In [155]:
NUM_FOLDS = 1
TEST_RATIO = 0.2
TRAIN_RATIO = 0.8 * 0.7
VALID_RATIO = 0.8 * 0.3

if SPLIT_PATH and os.path.exists(SPLIT_PATH):
    splits = joblib.load(SPLIT_PATH)
else:
    x = np.array(wsi_names)
    with open(LABELSPATH, 'r') as f:
        labels_dict = json.load(f)
    print(labels_dict)
    y = np.array([labels_dict[wsi_name+'.json'] for wsi_name in wsi_names])
    splits = stratified_split(x, y, TRAIN_RATIO, VALID_RATIO, TEST_RATIO, NUM_FOLDS)
    joblib.dump(splits, SPLIT_PATH)


In [175]:


# # we must define the function after training/loading
# def nodes_preproc_func(node_features: np.ndarray) -> np.ndarray:
#     """Pre-processing function for nodes."""
#     return node_scaler.transform(node_features)
nodes_preproc_func = None


splits = joblib.load(SPLIT_PATH)
loader_kwargs = {
    "num_workers": 6,
    "batch_size": 32,
}
# arch_kwargs = {
#     "dim_features": NUM_NODE_FEATURES,
#     "dim_target": NCLASSES,
#     "layers": [32, 32, 16, 8],
#     "dropout": 0.3,
#     "pooling": "mean",
#     "conv": "EdgeConv",
#     "aggr": "max",
# }


arch_kwargs = {
    "dim_features": NUM_NODE_FEATURES,
    "dim_target": NCLASSES,
    "layers": [64, 32, 8],
    "dropout": 0.3,
    "pooling": "mean",
    "conv": "EdgeConv",
    "aggr": "max",
}
optim_kwargs = {
    "lr": 5.0e-4,
    "weight_decay": 1.0e-4,
}

NUM_EPOCHS = 100
# if not MODEL_DIR.exists() or True:
for split_idx, split in enumerate(splits):
        new_split = {
            "train": split["train"],
            "infer-train": split["train"],
            "infer-valid-A": split["valid"],
            "infer-valid-B": split["test"],
        }
        MODEL_DIR = Path(MODEL_DIR) 
        split_save_dir = MODEL_DIR / f"{split_idx:02d}/"
        rm_n_mkdir(split_save_dir)
        reset_logging(split_save_dir)
        run_once(
            new_split,
            NUM_EPOCHS,
            save_dir=split_save_dir,
            arch_kwargs=arch_kwargs,
            loader_kwargs=loader_kwargs,
            optim_kwargs=optim_kwargs,
            on_gpu=ON_GPU,
        )

|2023-11-20|23:25:28.051| [INFO] EPOCH: 000


SlideGraphArch(
  (first_h): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (nns): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=128, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=64, out_features=8, bias=True)
      (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (convs): ModuleList(
    (0): EdgeConv(nn=Sequential(
      (0): Linear(in_features=128, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    ))
    (1): EdgeConv(nn=Sequential(
      (0): Linear(in_features=64, out_features=8, bias=True)
      (1): BatchNorm1d(8, eps=1e-05, mom

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:00<00:00, 13.30it/s]
|2023-11-20|23:25:28.580| [INFO] train-EMA-loss: 1.325166
|2023-11-20|23:25:28.960| [INFO] infer-train-accuracy: 0.331878
|2023-11-20|23:25:29.204| [INFO] infer-valid-A-accuracy: 0.346154
|2023-11-20|23:25:29.497| [INFO] infer-valid-B-accuracy: 0.343750
|2023-11-20|23:25:29.721| [INFO] EPOCH: 001


best_score {'infer-train-accuracy': 0, 'infer-valid-A-accuracy': 0, 'infer-valid-B-accuracy': 0}


...

best_score {'infer-train-accuracy': 0.9737991266375546, 'infer-valid-A-accuracy': 0.6153846153846154, 'infer-valid-B-accuracy': 0.53125}


100%|██████████| 7/7 [00:00<00:00, 13.49it/s]
|2023-11-20|23:28:19.831| [INFO] train-EMA-loss: 0.250289
|2023-11-20|23:28:20.203| [INFO] infer-train-accuracy: 0.973799
|2023-11-20|23:28:20.447| [INFO] infer-valid-A-accuracy: 0.500000
|2023-11-20|23:28:20.718| [INFO] infer-valid-B-accuracy: 0.500000
|2023-11-20|23:28:21.041| [INFO] EPOCH: 099


best_score {'infer-train-accuracy': 0.9737991266375546, 'infer-valid-A-accuracy': 0.6153846153846154, 'infer-valid-B-accuracy': 0.53125}


100%|██████████| 7/7 [00:00<00:00, 14.08it/s]
|2023-11-20|23:28:21.540| [INFO] train-EMA-loss: 0.395904
|2023-11-20|23:28:21.924| [INFO] infer-train-accuracy: 0.986900
|2023-11-20|23:28:22.155| [INFO] infer-valid-A-accuracy: 0.576923
|2023-11-20|23:28:22.416| [INFO] infer-valid-B-accuracy: 0.453125


best_score {'infer-train-accuracy': 0.9868995633187773, 'infer-valid-A-accuracy': 0.6153846153846154, 'infer-valid-B-accuracy': 0.53125}


In [164]:

arch_kwargs = {
    "dim_features": NUM_NODE_FEATURES,
    "dim_target": NCLASSES,
    "layers": [64, 32, 8],
    "dropout": 0.3,
    "pooling": "mean",
    "conv": "EdgeConv",
    "aggr": "max",
}
model = SlideGraphArch(**arch_kwargs)
model

SlideGraphArch(
  (first_h): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (nns): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=128, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=64, out_features=8, bias=True)
      (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (convs): ModuleList(
    (0): EdgeConv(nn=Sequential(
      (0): Linear(in_features=128, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    ))
    (1): EdgeConv(nn=Sequential(
      (0): Linear(in_features=64, out_features=8, bias=True)
      (1): BatchNorm1d(8, eps=1e-05, mom